In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
from sqlalchemy import create_engine
import time
from sklearn.preprocessing import StandardScaler, normalize, RobustScaler, MinMaxScaler, FunctionTransformer
from scipy.stats import boxcox
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import scipy.cluster.hierarchy as hc
from sklearn.cluster import AgglomerativeClustering
from haversine import haversine, Unit

In [2]:
number1 = pd.read_csv('/home/subin/MyGitProject/hsj-14/1번.csv')
number1

<ipython-input-2-4b5f40eb9d24>:1: DtypeWarning: Columns (5,6,7,11,12,13,14,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  number1 = pd.read_csv('/home/subin/MyGitProject/hsj-14/1번.csv')


,Unnamed: 0,시도,시군구,읍면동,complex_name,dong,ho,official_price,old_addr_idx,pk,...,위도,평수,위도+경도,closest_sub,closest_high,school_1km,area,floor,actual,거래날짜
0,0,blank,blank,blank,blank,blank,blank,blank,blank,blank,...,blank,blank,blank,blank,blank,blank,blank,blank,blank,blank
1,0,강원도,강릉시,강동면,정동진우리하우스텔1,단일,201,26000000.0,4215034030_00219_00000_apartment,20160538_0001_0001,...,37.676072,6.8030303030303,37.676072129.014238,2.35646044024309,2.11703734256027,0.0,7.0,2.0,NaN,NaN
2,1,강원도,강릉시,강동면,정동진우리하우스텔1,단일,202,31500000.0,4215034030_00219_00000_apartment,20160538_0001_0002,...,37.676072,8.28787878787879,37.676072129.014238,2.35646044024309,2.11703734256027,0.0,8.0,2.0,NaN,NaN
3,2,강원도,강릉시,강동면,정동진우리하우스텔1,단일,203,31500000.0,4215034030_00219_00000_apartment,20160538_0001_0003,...,37.676072,8.28787878787879,37.676072129.014238,2.35646044024309,2.11703734256027,0.0,8.0,2.0,NaN,NaN
4,3,강원도,강릉시,강동면,정동진우리하우스텔1,단일,204,31500000.0,4215034030_00219_00000_apartment,20160538_0001_0004,...,37.676072,8.28787878787879,37.676072129.014238,2.35646044024309,2.11703734256027,0.0,8.0,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999996,99995,경기도,부천시,도당동,동신,B,401,201000000.0,4119010400_00266_00003_apartment,00007025_0002_0028,...,37.512779,18.087879,37.512779126.784505,1.037948,0.709002,5.0,18.0,4.0,182500000.0,2018-01-08
9999997,99996,경기도,부천시,도당동,동신,B,401,201000000.0,4119010400_00266_00003_apartment,00007025_0002_0028,...,37.512779,18.087879,37.512779126.784505,1.037948,0.709002,5.0,18.0,4.0,265000000.0,2021-01-05
9999998,99997,경기도,부천시,도당동,동신,B,402,201000000.0,4119010400_00266_00003_apartment,00007025_0002_0029,...,37.512779,18.087879,37.512779126.784505,1.037948,0.709002,5.0,18.0,4.0,182500000.0,2018-01-08
9999999,99998,경기도,부천시,도당동,동신,B,402,201000000.0,4119010400_00266_00003_apartment,00007025_0002_0029,...,37.512779,18.087879,37.512779126.784505,1.037948,0.709002,5.0,18.0,4.0,265000000.0,2021-01-05


In [6]:
area_floor

,pk,area,floor
0,00010892_0001_0001,42,2
1,00010892_0001_0002,42,2
2,00010892_0001_0003,28,2
3,00010892_0001_0004,28,2
4,00010892_0001_0005,28,2
...,...,...,...
412317,20431058_0001_0014,26,8
412318,20431058_0001_0015,26,9
412319,20431058_0001_0016,26,9
412320,20431058_0001_0017,22,10


In [7]:
start_time = time.time()
con = pymysql.connect(host='localhost', user='jiwoo', password='1234', db='TESTDB', charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)
cur = con.cursor()
sql = "SELECT * FROM `대전광역시공시지가만 preprocessing`"
cur.execute(sql)
rows = cur.fetchall()
con.close()

In [8]:
total = pd.DataFrame(rows)

In [9]:
total

,시도,시군구,읍면동,complex_name,dong,ho,official_price,old_addr_idx,pk,updated,경도,리,위도,평수,위도+경도,subway_id,closest_sub,school_id,closest_high,school_1km
0,대전광역시,대덕구,대화동,금강,1,101,110000000.0,3023010200_00027_00007_apartment,00011718_0001_0001,2022-01-01,127.412043,None,36.365960,22.287879,36.36596127.412043,4.19민주묘지,4.114766,(가칭)시화MTV초등학교,0.368736,3.0
1,대전광역시,대덕구,대화동,금강,1,102,110000000.0,3023010200_00027_00007_apartment,00011718_0001_0002,2022-01-01,127.412043,None,36.365960,22.287879,36.36596127.412043,4.19민주묘지,4.114766,(가칭)시화MTV초등학교,0.368736,3.0
2,대전광역시,대덕구,대화동,금강,1,103,100000000.0,3023010200_00027_00007_apartment,00011718_0001_0003,2022-01-01,127.412043,None,36.365960,17.048485,36.36596127.412043,4.19민주묘지,4.114766,(가칭)시화MTV초등학교,0.368736,3.0
3,대전광역시,대덕구,대화동,금강,1,104,100000000.0,3023010200_00027_00007_apartment,00011718_0001_0004,2022-01-01,127.412043,None,36.365960,17.048485,36.36596127.412043,4.19민주묘지,4.114766,(가칭)시화MTV초등학교,0.368736,3.0
4,대전광역시,대덕구,대화동,금강,1,105,103000000.0,3023010200_00027_00007_apartment,00011718_0001_0005,2022-01-01,127.412043,None,36.365960,17.048485,36.36596127.412043,4.19민주묘지,4.114766,(가칭)시화MTV초등학교,0.368736,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,대전광역시,중구,None,(645-10),1,302,78700000.0,3014011900_00645_00010_apartment,20293767_0001_0004,2021-01-01,127.413932,None,36.301204,18.266667,36.3012045127.41393246153848,4.19민주묘지,2.551867,(가칭)시화MTV초등학교,1.312272,0.0
412318,대전광역시,중구,None,(645-10),1,401,78700000.0,3014011900_00645_00010_apartment,20293767_0001_0005,2021-01-01,127.413932,None,36.301204,18.266667,36.3012045127.41393246153848,4.19민주묘지,2.551867,(가칭)시화MTV초등학교,1.312272,0.0
412319,대전광역시,중구,None,(645-10),1,402,78700000.0,3014011900_00645_00010_apartment,20293767_0001_0006,2021-01-01,127.413932,None,36.301204,18.266667,36.3012045127.41393246153848,4.19민주묘지,2.551867,(가칭)시화MTV초등학교,1.312272,0.0
412320,대전광역시,중구,None,(645-10),1,501,78700000.0,3014011900_00645_00010_apartment,20293767_0001_0007,2021-01-01,127.413932,None,36.301204,18.266667,36.3012045127.41393246153848,4.19민주묘지,2.551867,(가칭)시화MTV초등학교,1.312272,0.0


In [2]:
df1 = pd.read_csv('/home/subin/다운로드/여피/0308 업무-20230310T030110Z-001/0308 업무/complex_20230308 (사본).csv', header=None)
df1.columns = ['old_addr_idx', 'complex_name', 'property_type','시도', '시군구',
               '읍면동','리','road_name','road_code','admin_dong_code','postal_code','jibun_main','jibun_sub','updated']
df1.drop(['road_name','road_code','admin_dong_code','postal_code','jibun_main','jibun_sub', 'updated'], axis=1, inplace=True)

df1['시군구'] = df1['시군구'].str.split().str[0]
df1 = df1[df1['property_type']!='officetel']
df1.drop(['property_type'], axis=1, inplace=True)

In [3]:
df2 = pd.read_csv('/home/subin/다운로드/여피/0308 업무-20230310T030110Z-001/0308 업무/property_20230308.csv', header=None)
df2.columns = ['pk','old_addr_idx','dong','ho','official_price','net_leasable_area','updated']

In [4]:
app_apt = df2.merge(df1, on='old_addr_idx', how='left')
pd.options.display.max_columns = None
app_apt['평수'] = app_apt['net_leasable_area'] / 3.3
app_apt.drop(['net_leasable_area'], axis=1, inplace=True)
app_apt.replace('\\N', np.nan, inplace=True)
app_apt['시군구'] = app_apt['시군구'].fillna('세종특별자치시')

In [112]:
# 기존 : 공시지가 정보가 모두 존재하는 쪽에 맞추지 않고 중복되는 것만 구했었음.
app_apt = df2.merge(df1, on='old_addr_idx')
app_apt

,pk,old_addr_idx,dong,ho,official_price,updated,complex_name,시도,시군구,읍면동,리,평수
0,00000001_0001_0001,1111010100_00004_00007_apartment,단일,101,1249000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,63.978788
1,00000001_0001_0002,1111010100_00004_00007_apartment,단일,102,1076000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,63.396970
2,00000001_0001_0003,1111010100_00004_00007_apartment,단일,201,1140000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,62.996970
3,00000001_0001_0004,1111010100_00004_00007_apartment,단일,202,1249000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,65.178788
4,00000001_0001_0005,1111010100_00004_00007_apartment,단일,301,1332000000,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN,66.463636
...,...,...,...,...,...,...,...,...,...,...,...,...
14496524,20432545_0001_0012,2647010200_01288_00036_apartment,A,404,43100000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.539394
14496525,20432545_0001_0013,2647010200_01288_00036_apartment,A,502,43300000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.630303
14496526,20432545_0001_0014,2647010200_01288_00036_apartment,A,504,42400000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.539394
14496527,20432545_0001_0015,2647010200_01288_00036_apartment,A,501,42400000,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,NaN,5.539394


In [5]:
# null 값이 있어도 공시지가 정보가 비는 경우는 없음!
app_apt = df2.merge(df1, on='old_addr_idx', how='left')
app_apt

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리
0,00000001_0001_0001,1111010100_00004_00007_apartment,단일,101,1249000000,211.13,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
1,00000001_0001_0002,1111010100_00004_00007_apartment,단일,102,1076000000,209.21,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
2,00000001_0001_0003,1111010100_00004_00007_apartment,단일,201,1140000000,207.89,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
3,00000001_0001_0004,1111010100_00004_00007_apartment,단일,202,1249000000,215.09,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
4,00000001_0001_0005,1111010100_00004_00007_apartment,단일,301,1332000000,219.33,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
15362585,999_1(단일)_3_316,1165010800_01542_00006_officetel,단일,316,347655000,49.97,2021-01-01,NaN,NaN,NaN,NaN,NaN
15362586,999_1(단일)_3_317,1165010800_01542_00006_officetel,단일,317,304612000,43.97,2021-01-01,NaN,NaN,NaN,NaN,NaN
15362587,999_1(단일)_3_318,1165010800_01542_00006_officetel,단일,318,304612000,43.97,2021-01-01,NaN,NaN,NaN,NaN,NaN
15362588,999_1(단일)_3_319,1165010800_01542_00006_officetel,단일,319,304612000,43.97,2021-01-01,NaN,NaN,NaN,NaN,NaN


In [6]:
# 공시지가 정보는 모두 있지만 시도 정보가 들어있지 않은 경우가 약 86만개 존재
app_apt.isnull().sum()

pk                          0
old_addr_idx                0
dong                        0
ho                          3
official_price              0
net_leasable_area           0
updated                     0
complex_name           866061
시도                     866061
시군구                    986570
읍면동                   1183541
리                    12250570
dtype: int64

In [6]:
app_apt_tot = app_apt[~app_apt['old_addr_idx'].str.contains('officetel')]

In [8]:
# 그렇지만 오피스텔을 제외하니 행 수 자체는 다시 같아졌음.
app_apt_tot

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리
0,00000001_0001_0001,1111010100_00004_00007_apartment,단일,101,1249000000,211.13,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
1,00000001_0001_0002,1111010100_00004_00007_apartment,단일,102,1076000000,209.21,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
2,00000001_0001_0003,1111010100_00004_00007_apartment,단일,201,1140000000,207.89,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
3,00000001_0001_0004,1111010100_00004_00007_apartment,단일,202,1249000000,215.09,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
4,00000001_0001_0005,1111010100_00004_00007_apartment,단일,301,1332000000,219.33,2021-01-01,신구파인힐,서울특별시,종로구,청운동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14641574,20432545_0001_0012,2647010200_01288_00036_apartment,A,404,43100000,18.28,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,\N
14641575,20432545_0001_0013,2647010200_01288_00036_apartment,A,502,43300000,18.58,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,\N
14641576,20432545_0001_0014,2647010200_01288_00036_apartment,A,504,42400000,18.28,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,\N
14641577,20432545_0001_0015,2647010200_01288_00036_apartment,A,501,42400000,18.28,2022-01-01,더페이스(A동),부산광역시,연제구,연산동,\N


In [9]:
# 그리고 아파트에서는 시도 정보가 누락되는 경우가 없음!
app_apt_tot.isnull().sum()

pk                          0
old_addr_idx                0
dong                        0
ho                          3
official_price              0
net_leasable_area           0
updated                     0
complex_name                0
시도                          0
시군구                    120509
읍면동                    317480
리                    11384509
dtype: int64

In [113]:
# 이전 버전
daejeon = app_apt[app_apt['시도']=='대전광역시']
daejeon

,pk,old_addr_idx,dong,ho,official_price,updated,complex_name,시도,시군구,읍면동,리,평수
3032783,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,42.500000
3032784,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,41.733333
3032785,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032786,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032787,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
...,...,...,...,...,...,...,...,...,...,...,...,...
14493165,20431058_0001_0014,3014011000_00233_00043_apartment,단일,802,215000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,25.747364
14493166,20431058_0001_0015,3014011000_00233_00043_apartment,단일,901,215000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,25.747364
14493167,20431058_0001_0016,3014011000_00233_00043_apartment,단일,902,215000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,25.747364
14493168,20431058_0001_0017,3014011000_00233_00043_apartment,단일,1001,250000000,2022-01-01,더퍼스트뷰,대전광역시,중구,대사동,NaN,22.255485


In [7]:
# 그치만 현재 버전과 차이는 없는 것으로 드러남.
daejeon = app_apt_tot[app_apt_tot['시도']=='대전광역시']
daejeon

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리
2955218,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,140.25,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN
2955219,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,137.72,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN
2955220,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN
2955221,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN
2955222,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14641483,20431775_0001_0040,3011011700_00135_00002_apartment,3,504,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N
14641484,20431775_0001_0041,3011011700_00135_00002_apartment,3,505,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N
14641485,20431775_0001_0042,3011011700_00135_00002_apartment,3,506,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N
14641486,20431775_0001_0043,3011011700_00135_00002_apartment,3,507,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N


In [8]:
daejeon=daejeon.fillna('NONE')

In [9]:
daejeon['호']=daejeon['ho'].astype(str)
daejeon['호int'] = daejeon['호'].apply(lambda x: int(x) if x.isnumeric() else 0)
daejeon['호str'] = daejeon['호'].apply(lambda x: np.nan if x.isnumeric() else 0 if x.isdigit() else x)

start = 100
step = 100
num_steps = 83
ranges1 = [(start + step * i, start + step * i + 10, i+1) for i in range(num_steps)]
labels1 = [r[2] for r in ranges1]
bins1 = [r[0] for r in ranges1] + [ranges1[-1][1]]

bins = bins1
labels = labels1

daejeon['층'] = pd.cut(daejeon['호int'], bins=bins, labels=labels, include_lowest=True, right=False)

floorSplit = daejeon[daejeon['호str'].str.contains('층')==True]

floorSplit['호str'] = floorSplit['호str'].str.split('층').str[0]

floorSplit['호int'] = floorSplit['호str'].apply(lambda x: int(x) if x.isnumeric() else 0)



<ipython-input-9-e6c96394cc78>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floorSplit['호str'] = floorSplit['호str'].str.split('층').str[0]
<ipython-input-9-e6c96394cc78>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floorSplit['호int'] = floorSplit['호str'].apply(lambda x: int(x) if x.isnumeric() else 0)


In [10]:
start = 100
step = 100
num_steps = 16
ranges = [(start + step * i, start + step * i + 10, i+1) for i in range(num_steps)]

labels = [r[2] for r in ranges]
bins = [r[0] for r in ranges] + [ranges[-1][1]]

floorSplit['층'] = pd.cut(floorSplit['호int'], bins=bins, labels=labels, include_lowest=True, right=False)

<ipython-input-10-9318b6f1daca>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floorSplit['층'] = pd.cut(floorSplit['호int'], bins=bins, labels=labels, include_lowest=True, right=False)


In [11]:
start1 = 1
step1 = 1
num_steps1 = 90
ranges2 = [(start1 + step1 * i, start1 + step1 * i + 0.1, i+1) for i in range(num_steps1)]
labels2 = [r[2] for r in ranges2]
bins2 = [r[0] for r in ranges2] + [ranges2[-1][1]]

# concatenate the two lists of bin edges and labels
bins = bins2
labels = labels2
# convert the '호int' column to a categorical column
floorSplit['층2'] = pd.cut(floorSplit['호int'], bins=bins, labels=labels, include_lowest=True, right=False)

<ipython-input-11-1196a492ffd3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floorSplit['층2'] = pd.cut(floorSplit['호int'], bins=bins, labels=labels, include_lowest=True, right=False)


In [12]:
codes = floorSplit['층'].cat.codes
codes2 = floorSplit['층2'].cat.codes
floorSplit['codes'] = codes.reindex(floorSplit.index)
floorSplit['codes2'] = codes2.reindex(floorSplit.index)
floorSplit['codes'].replace(-1, np.nan, inplace=True)
floorSplit['codes2'].replace(-1, np.nan, inplace=True)
floorSplit['codes'] = floorSplit['codes'] + 1
floorSplit['codes2'] = floorSplit['codes2'] + 1
floorSplit['codes'] = floorSplit['codes'].fillna(floorSplit['codes2'])
floorSplit.drop(['층', '층2', 'codes2'], axis=1, inplace=True)

<ipython-input-12-493d9e1025b8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floorSplit['codes'] = codes.reindex(floorSplit.index)
<ipython-input-12-493d9e1025b8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floorSplit['codes2'] = codes2.reindex(floorSplit.index)
<ipython-input-12-493d9e1025b8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [13]:
check = floorSplit[floorSplit['codes'].isnull()==True]

jiha = check[check['호str'].str.contains('지하')==True]
jiha['codes'] = -1
jiha = jiha[['pk', 'codes']]
check = check.merge(jiha, on=['pk'], how='left')
check = check.drop('codes_x', axis=1)
check = check.rename(columns={'codes_y':'codes'})

jisang = check[check['호str'].str.contains('지상')]
jisang['호str'] = jisang['호str'].str.split('지상').str[1]
jisang['codes'] = jisang['호str'].apply(lambda x: int(x) if x.isnumeric() else 0)

mask = (jisang['pk']=='11999085_0001_0001')
jisang.loc[mask, 'codes']=1
mask = (jisang['pk']=='12148950_0001_0002')
jisang.loc[mask, 'codes']=2
mask = (jisang['pk']=='12148950_0001_0004')
jisang.loc[mask, 'codes']=3
mask = (jisang['pk']=='12148950_0001_0005')
jisang.loc[mask, 'codes']=3
mask = (jisang['pk']=='12148950_0001_0006')
jisang.loc[mask, 'codes']=4

jisang = jisang[['pk', 'codes']]

check = check.merge(jisang, on=['pk'], how='left')
check['codes_x'] = check['codes_x'].fillna(check['codes_y'])
check = check.rename(columns={'codes_x':'codes'})
check.drop('codes_y', axis=1, inplace=True)

justJi = check[check['호str'].str.contains('지')&~(check['호str'].str.contains('지하'))&~(check['호str'].str.contains('지상'))]
justJi['codes'] = -1

justJi = justJi[['pk', 'codes']]

check = check.merge(justJi, on=['pk'], how='left')
check['codes_x'] = check['codes_x'].fillna(check['codes_y'])
check = check.rename(columns={'codes_x':'codes'})
check.drop('codes_y', axis=1, inplace=True)

<ipython-input-13-a9c882ab0663>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jiha['codes'] = -1
<ipython-input-13-a9c882ab0663>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  justJi['codes'] = -1


In [14]:
dong = check[(check['호str'].str.contains('동')==True) & (check['호str'].str.contains('지')!=True)]
dong['호str'] = dong['호str'].str.split('동').str[1]

mask = (dong['호str'].str.contains('3'))
dong.loc[mask, 'codes'] = 3
mask = (dong['호str'].str.contains('4'))
dong.loc[mask, 'codes'] = 4
mask = (dong['호str'].str.contains('일'))
dong.loc[mask, 'codes'] = 1
mask = (dong['호str'].str.contains('이'))
dong.loc[mask, 'codes'] = 2
mask = (dong['호str'].str.contains('삼'))
dong.loc[mask, 'codes'] = 3
mask = (dong['호str'].str.contains('사'))
dong.loc[mask, 'codes'] = 4
mask = (dong['호str'].str.contains('5'))
dong.loc[mask, 'codes'] = 5

mask = (dong['호str'].str.contains('1')) & (~dong['호str'].str.contains('2'))
dong.loc[mask, 'codes'] = 1
mask = (dong['호str'].str.contains('2')) & (~dong['호str'].str.contains('1'))
dong.loc[mask, 'codes'] = 2

mask = (dong['호str']=='10')
dong.loc[mask, 'codes'] = 10
mask = (dong['호str']=='11')
dong.loc[mask, 'codes'] = 11
mask = (dong['호str']=='12')
dong.loc[mask, 'codes'] = 12
mask = (dong['호str']=='13')
dong.loc[mask, 'codes'] = 13
mask = (dong['호str']=='14')
dong.loc[mask, 'codes'] = 14
mask = (dong['호str']=='15')
dong.loc[mask, 'codes'] = 15

mask = (dong['호str']=='2호1')
dong.loc[mask, 'codes'] = 1
mask = (dong['호str']=='1호2')
dong.loc[mask, 'codes'] = 2

dong = dong[['pk', 'codes']]

check = check.merge(dong, on='pk', how='left')
check['codes_x'] = check['codes_x'].fillna(check['codes_y'])
check = check.rename(columns={'codes_x':'codes'})
check.drop('codes_y', axis=1, inplace=True)

<ipython-input-14-9432fbcc790e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dong['호str'] = dong['호str'].str.split('동').str[1]


In [15]:
rest = check[check['codes'].isnull()==True]

regex = r'\D+(\d+)$'
rest['codes'] = rest['호str'].str.extract(regex)

rest_of_rest = rest[rest['codes'].isnull()==True]

mask = (rest_of_rest['호str'].str.contains('일'))
rest_of_rest.loc[mask, 'codes'] = 1
mask = (rest_of_rest['호str'].str.contains('이'))
rest_of_rest.loc[mask, 'codes'] = 2
mask = (rest_of_rest['호str'].str.contains('사'))
rest_of_rest.loc[mask, 'codes'] = 4
mask = (rest_of_rest['호str'].str.contains('삼'))
rest_of_rest.loc[mask, 'codes'] = 3
mask = (rest_of_rest['호str'].str.contains('오'))
rest_of_rest.loc[mask, 'codes'] = 5
mask = (rest_of_rest['호str'].str.contains('육'))
rest_of_rest.loc[mask, 'codes'] = 6
mask = (rest_of_rest['호str'].str.contains('칠'))
rest_of_rest.loc[mask, 'codes'] = 7

rest_of_rest_bok = rest_of_rest[rest_of_rest['codes'].isnull()==True]
rest_of_rest_bok = rest_of_rest[rest_of_rest['호str'].str.contains('복')]
rest_of_rest_bok['호str']= rest_of_rest_bok['호str'].str.split('복').str[0]

rest_of_rest_bok['numeric'] = rest_of_rest_bok['호str'].str.extract('(\d+)')
rest_of_rest_bok['numeric'] = rest_of_rest_bok['numeric'].fillna(0)
rest_of_rest_bok['numeric'] = rest_of_rest_bok['numeric'].astype(int)

start = 100
step = 100
num_steps = 16
ranges = [(start + step * i, start + step * i + 10, i+1) for i in range(num_steps)]

labels = [r[2] for r in ranges]
bins = [r[0] for r in ranges] + [ranges[-1][1]]

rest_of_rest_bok['층'] = pd.cut(rest_of_rest_bok['numeric'], bins=bins, labels=labels, include_lowest=True, right=False)

<ipython-input-15-88e8d8d98f9b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest['codes'] = rest['호str'].str.extract(regex)


In [16]:
codes3 = rest_of_rest_bok['층'].cat.codes
rest_of_rest_bok['codes3'] = codes3.reindex(rest_of_rest_bok.index)
rest_of_rest_bok['codes3'].replace(-1, np.nan, inplace=True)
rest_of_rest_bok['codes3'] = rest_of_rest_bok['codes3'] + 1
rest_of_rest_bok.drop(['codes', 'numeric', '층'], axis=1, inplace=True)
rest_of_rest_bok = rest_of_rest_bok.rename(columns={'codes3':'codes'})
rest_of_rest_bok = rest_of_rest_bok[['pk', 'codes']]

rest_of_rest = rest_of_rest.merge(rest_of_rest_bok, on='pk', how='left')
rest_of_rest['codes_x'] = rest_of_rest['codes_x'].fillna(rest_of_rest['codes_y'])
rest_of_rest = rest_of_rest.rename(columns={'codes_x':'codes'})
rest_of_rest.drop('codes_y', axis=1, inplace=True)

mask = (rest_of_rest['codes']==0)
rest_of_rest.loc[mask, 'codes'] = 3

<ipython-input-16-1b9d5dade8fd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_of_rest_bok.drop(['codes', 'numeric', '층'], axis=1, inplace=True)


In [17]:
rest_of_rest_ = rest_of_rest[rest_of_rest['codes'].isnull()==True]

mask = (rest_of_rest_['ho'].str.contains('별층'))
rest_of_rest_.loc[mask, 'codes'] = -1
mask = (rest_of_rest_['ho'].str.contains('비층'))
rest_of_rest_.loc[mask, 'codes'] = -1
mask = (rest_of_rest_['ho'].str.contains('B'))
rest_of_rest_.loc[mask, 'codes'] = -1
mask = (rest_of_rest_['ho'].str.contains('401'))
rest_of_rest_.loc[mask, 'codes'] = 4
mask = (rest_of_rest_['ho'].str.contains('402'))
rest_of_rest_.loc[mask, 'codes'] = 4
mask = (rest_of_rest_['ho'].str.contains('201'))
rest_of_rest_.loc[mask, 'codes'] = 2
mask = (rest_of_rest_['ho'].str.contains('202'))
rest_of_rest_.loc[mask, 'codes'] = 2

mask = (rest_of_rest_['pk']=='12085035_0001_0004')
rest_of_rest_.loc[mask, 'codes'] = 4

rest_of_rest_ = rest_of_rest_[['pk', 'codes']]

rest_of_rest = rest_of_rest.merge(rest_of_rest_, on='pk', how='left')
rest_of_rest['codes_x'] = rest_of_rest['codes_x'].fillna(rest_of_rest['codes_y'])
rest_of_rest = rest_of_rest.rename(columns={'codes_x':'codes'})
rest_of_rest.drop('codes_y', axis=1, inplace=True)

rest_of_rest = rest_of_rest[['pk', 'codes']]
rest = rest.merge(rest_of_rest, on='pk', how='left')
rest['codes_x'] = rest['codes_x'].fillna(rest['codes_y'])
rest = rest.rename(columns={'codes_x':'codes'})
rest.drop('codes_y', axis=1, inplace=True)

rest = rest[['pk', 'codes']]
check = check.merge(rest, on='pk', how='left')
check['codes_x'] = check['codes_x'].fillna(check['codes_y'])
check = check.rename(columns={'codes_x':'codes'})
check.drop('codes_y', axis=1, inplace=True)

check = check[['pk', 'codes']]
floorSplit = floorSplit.merge(check, on='pk', how='left')
floorSplit['codes_x'] = floorSplit['codes_x'].fillna(floorSplit['codes_y'])
floorSplit = floorSplit.rename(columns={'codes_x':'codes'})
floorSplit.drop('codes_y', axis=1, inplace=True)

mask = (floorSplit['pk']=='12030365_0001_0005')
floorSplit.loc[mask, 'codes'] = 3
mask = (floorSplit['pk']=='12035834_0001_0003')
floorSplit.loc[mask, 'codes'] = 3

In [18]:
codes4 = daejeon['층'].cat.codes
daejeon['codes'] = codes4.reindex(daejeon.index)
daejeon['codes'] = daejeon['codes'] + 1
daejeon['codes'].replace(0, np.nan, inplace=True)

floorSplit = floorSplit[['pk', 'codes']]
daejeon = daejeon.merge(floorSplit, on='pk', how='left')
daejeon['codes_x'] = daejeon['codes_x'].fillna(daejeon['codes_y'])
daejeon = daejeon.rename(columns={'codes_x':'codes'})
daejeon.drop('codes_y', axis=1, inplace=True)

In [19]:
last = daejeon[daejeon['codes'].isnull()==True]
last.drop(['호int', '호str', '층'], axis=1, inplace=True)

<ipython-input-19-224767ef5c18>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last.drop(['호int', '호str', '층'], axis=1, inplace=True)


In [20]:
jiha2 = last[last['호'].str.contains('지하')==True]
jiha2['codes'] = -1
jiha2 = jiha2[['pk', 'codes']]
last = last.merge(jiha2, on=['pk'], how='left')
last = last.drop('codes_x', axis=1)
last = last.rename(columns={'codes_y':'codes'})

<ipython-input-20-c4a92d2f4a90>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jiha2['codes'] = -1


In [21]:
jisang2 = last[last['호'].str.contains('지상')]
jisang2['호'] = jisang2['호'].str.split('지상').str[1]

In [22]:
jisang2['numeric'] = jisang2['호'].str.extract('(\d+)')
jisang2['numeric'] = jisang2['numeric'].fillna(0)
jisang2['numeric'] = jisang2['numeric'].astype(int)

start = 100
step = 100
num_steps = 18
ranges = [(start + step * i, start + step * i + 10, i+1) for i in range(num_steps)]

labels = [r[2] for r in ranges]
bins = [r[0] for r in ranges] + [ranges[-1][1]]

jisang2['층'] = pd.cut(jisang2['numeric'], bins=bins, labels=labels, include_lowest=True, right=False)

start1 = 1
step1 = 1
num_steps1 = 90
ranges2 = [(start1 + step1 * i, start1 + step1 * i + 0.1, i+1) for i in range(num_steps1)]
labels2 = [r[2] for r in ranges2]
bins2 = [r[0] for r in ranges2] + [ranges2[-1][1]]

# concatenate the two lists of bin edges and labels
bins = bins2
labels = labels2
# convert the 'numeric' column to a categorical column
jisang2['층2'] = pd.cut(jisang2['numeric'], bins=bins, labels=labels, include_lowest=True, right=False)

codes = jisang2['층'].cat.codes
codes2 = jisang2['층2'].cat.codes
jisang2['codes'] = codes.reindex(jisang2.index)
jisang2['codes2'] = codes2.reindex(jisang2.index)
jisang2['codes'].replace(-1, np.nan, inplace=True)
jisang2['codes2'].replace(-1, np.nan, inplace=True)
jisang2['codes'] = jisang2['codes'] + 1
jisang2['codes2'] = jisang2['codes2'] + 1
jisang2['codes'] = jisang2['codes'].fillna(jisang2['codes2'])
jisang2.drop(['층', '층2', 'codes2', 'numeric'], axis=1, inplace=True)

mask = (jisang2['pk']=='20031264_0001_0001')
jisang2.loc[mask, 'codes'] = 1

<ipython-input-22-4e5c74641632>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jisang2.drop(['층', '층2', 'codes2', 'numeric'], axis=1, inplace=True)


In [23]:
jisang2 = jisang2[['pk', 'codes']]
last = last.merge(jisang2, on='pk', how='left')
last['codes_x'] = last['codes_x'].fillna(last['codes_y'])
last = last.rename(columns={'codes_x':'codes'})
last.drop('codes_y', axis=1, inplace=True)

In [24]:
last = last[['pk', 'codes']]
last = last.dropna(subset='codes')
daejeon = daejeon.merge(last, on='pk', how='left')
daejeon['codes_x'] = daejeon['codes_x'].fillna(daejeon['codes_y'])
daejeon = daejeon.rename(columns={'codes_x':'codes'})
daejeon.drop('codes_y', axis=1, inplace=True)

In [25]:
last2 = daejeon[daejeon['codes'].isnull()==True]
last2['numeric'] = last2['호'].str.extract('(\d+)')
last2['numeric'] = last2['numeric'].fillna(0)
last2['numeric'] = last2['numeric'].astype(int)

start = 100
step = 100
num_steps = 50
ranges = [(start + step * i, start + step * i + 10, i+1) for i in range(num_steps)]

labels = [r[2] for r in ranges]
bins = [r[0] for r in ranges] + [ranges[-1][1]]

last2['층'] = pd.cut(last2['numeric'], bins=bins, labels=labels, include_lowest=True, right=False)

<ipython-input-25-a74c6130a6b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last2['numeric'] = last2['호'].str.extract('(\d+)')
<ipython-input-25-a74c6130a6b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last2['numeric'] = last2['numeric'].fillna(0)
<ipython-input-25-a74c6130a6b1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [26]:
mask = (last2['numeric']==0)
last2.loc[mask, 'numeric'] = 1

In [27]:
mask = (last2['호'].str.contains('B0'))
last2.loc[mask, 'numeric'] = -1

In [28]:
codes5 = last2['층'].cat.codes
last2['codes2'] = codes.reindex(last2.index)
last2['codes2'].replace(-1, np.nan, inplace=True)
last2['codes2'] = last2['codes2'] + 1
last2['codes'] = last2['codes'].fillna(last2['codes2'])
last2['codes'] = last2['codes'].fillna(last2['numeric'])
last2.drop(['층','codes2', 'numeric'], axis=1, inplace=True)

<ipython-input-28-924028124f2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last2['codes2'] = codes.reindex(last2.index)
<ipython-input-28-924028124f2b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last2['codes2'].replace(-1, np.nan, inplace=True)
<ipython-input-28-924028124f2b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [29]:
last2 = last2[['pk', 'codes']]
daejeon = daejeon.merge(last2, on='pk', how='left')
daejeon['codes_x'] = daejeon['codes_x'].fillna(daejeon['codes_y'])
daejeon = daejeon.rename(columns={'codes_x':'codes'})
daejeon.drop('codes_y', axis=1, inplace=True)

In [30]:
daejeon

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리,호,호int,호str,층,codes
0,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,140.25,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,201,201,NaN,2,2.0
1,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,137.72,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,202,202,NaN,2,2.0
2,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,203,203,NaN,2,2.0
3,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,204,204,NaN,2,2.0
4,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,205,205,NaN,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,20431775_0001_0040,3011011700_00135_00002_apartment,3,504,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,504,504,NaN,5,5.0
412318,20431775_0001_0041,3011011700_00135_00002_apartment,3,505,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,505,505,NaN,5,5.0
412319,20431775_0001_0042,3011011700_00135_00002_apartment,3,506,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,506,506,NaN,5,5.0
412320,20431775_0001_0043,3011011700_00135_00002_apartment,3,507,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,507,507,NaN,5,5.0


In [34]:
daejeon.isnull().sum()

pk                        0
old_addr_idx              0
dong                      0
ho                        0
official_price            0
net_leasable_area         0
updated                   0
complex_name              0
시도                        0
시군구                       0
읍면동                       0
리                         0
호                         0
호int                      0
호str                 407363
층                      7279
codes                     0
dtype: int64

In [31]:
daejeon['평수'] = daejeon['net_leasable_area']/3.3
daejeon['area'] = daejeon['평수'].round(0)

In [32]:
daejeon

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리,호,호int,호str,층,codes,평수,area
0,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,140.25,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,201,201,NaN,2,2.0,42.500000,42.0
1,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,137.72,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,202,202,NaN,2,2.0,41.733333,42.0
2,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,203,203,NaN,2,2.0,28.100000,28.0
3,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,204,204,NaN,2,2.0,28.100000,28.0
4,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NONE,205,205,NaN,2,2.0,28.100000,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,20431775_0001_0040,3011011700_00135_00002_apartment,3,504,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,504,504,NaN,5,5.0,17.854545,18.0
412318,20431775_0001_0041,3011011700_00135_00002_apartment,3,505,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,505,505,NaN,5,5.0,17.854545,18.0
412319,20431775_0001_0042,3011011700_00135_00002_apartment,3,506,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,506,506,NaN,5,5.0,17.854545,18.0
412320,20431775_0001_0043,3011011700_00135_00002_apartment,3,507,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,\N,507,507,NaN,5,5.0,17.854545,18.0


In [33]:
areaCode = pd.read_excel('/home/subin/다운로드/행정_법정동 중심좌표_동별.xlsx')
areaCode = areaCode[areaCode['코드종류']=='B']
areaCode.drop(['코드', '코드종류'], axis=1, inplace=True)
areaCode['시군구'] = areaCode['시군구'].str.split().str[0]
areaCode

,시도,시군구,읍면동,리,위도,경도
3546,서울특별시,종로구,청운동,NaN,37.587111,126.969069
3547,서울특별시,종로구,신교동,NaN,37.583911,126.968354
3548,서울특별시,종로구,궁정동,NaN,37.584381,126.971489
3549,서울특별시,종로구,효자동,NaN,37.582416,126.971670
3550,서울특별시,종로구,창성동,NaN,37.580363,126.972065
...,...,...,...,...,...,...
22214,제주특별자치도,서귀포시,표선면,하천리,33.348068,126.832421
22215,제주특별자치도,서귀포시,표선면,성읍리,33.393144,126.794822
22216,제주특별자치도,서귀포시,표선면,가시리,33.354784,126.771072
22217,제주특별자치도,서귀포시,표선면,세화리,33.321147,126.798867


In [34]:
daejeon_areaCode = areaCode[areaCode['시도']=='대전광역시']
daejeon_areaCode

,시도,시군구,읍면동,리,위도,경도
5009,대전광역시,동구,원동,NaN,36.327973,127.433507
5010,대전광역시,동구,인동,NaN,36.322645,127.437417
5011,대전광역시,동구,효동,NaN,36.316405,127.441211
5012,대전광역시,동구,천동,NaN,36.316328,127.445916
5013,대전광역시,동구,가오동,NaN,36.307571,127.454441
...,...,...,...,...,...,...
5177,대전광역시,대덕구,이현동,NaN,36.411749,127.464183
5178,대전광역시,대덕구,갈전동,NaN,36.427571,127.465360
5179,대전광역시,대덕구,삼정동,NaN,36.446946,127.466818
5180,대전광역시,대덕구,미호동,NaN,36.466118,127.470775


In [44]:
daejeon_areaCode.isnull().sum()

시도       0
시군구      0
읍면동      0
리      173
위도       0
경도       0
dtype: int64

In [35]:
daejeon = daejeon.replace('NONE', np.nan)
daejeon = daejeon.replace('\\N', np.nan)
daejeon

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리,호,호int,호str,층,codes,평수,area
0,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,140.25,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,201,201,NaN,2,2.0,42.500000,42.0
1,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,137.72,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,202,202,NaN,2,2.0,41.733333,42.0
2,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,203,203,NaN,2,2.0,28.100000,28.0
3,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,204,204,NaN,2,2.0,28.100000,28.0
4,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,205,205,NaN,2,2.0,28.100000,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,20431775_0001_0040,3011011700_00135_00002_apartment,3,504,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,504,504,NaN,5,5.0,17.854545,18.0
412318,20431775_0001_0041,3011011700_00135_00002_apartment,3,505,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,505,505,NaN,5,5.0,17.854545,18.0
412319,20431775_0001_0042,3011011700_00135_00002_apartment,3,506,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,506,506,NaN,5,5.0,17.854545,18.0
412320,20431775_0001_0043,3011011700_00135_00002_apartment,3,507,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,507,507,NaN,5,5.0,17.854545,18.0


In [58]:
daejeon.isnull().sum()

pk                        0
old_addr_idx              0
dong                      0
ho                        0
official_price            0
net_leasable_area         0
updated                   0
complex_name              0
시도                        0
시군구                       0
읍면동                    9343
리                    412322
호                         0
호int                      0
호str                 407363
층                      7279
codes                     0
area                      0
dtype: int64

In [36]:
korea = daejeon.merge(daejeon_areaCode, on=['시도', '시군구', '읍면동', '리'], how='left')
korea

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리,호,호int,호str,층,codes,평수,area,위도,경도
0,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,140.25,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,201,201,NaN,2,2.0,42.500000,42.0,36.331098,127.418073
1,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,137.72,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,202,202,NaN,2,2.0,41.733333,42.0,36.331098,127.418073
2,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,203,203,NaN,2,2.0,28.100000,28.0,36.331098,127.418073
3,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,204,204,NaN,2,2.0,28.100000,28.0,36.331098,127.418073
4,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,92.73,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,205,205,NaN,2,2.0,28.100000,28.0,36.331098,127.418073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,20431775_0001_0040,3011011700_00135_00002_apartment,3,504,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,504,504,NaN,5,5.0,17.854545,18.0,36.347593,127.425887
412318,20431775_0001_0041,3011011700_00135_00002_apartment,3,505,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,505,505,NaN,5,5.0,17.854545,18.0,36.347593,127.425887
412319,20431775_0001_0042,3011011700_00135_00002_apartment,3,506,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,506,506,NaN,5,5.0,17.854545,18.0,36.347593,127.425887
412320,20431775_0001_0043,3011011700_00135_00002_apartment,3,507,74500000,58.92,2022-01-01,시영아파트,대전광역시,동구,홍도동,NaN,507,507,NaN,5,5.0,17.854545,18.0,36.347593,127.425887


In [60]:
korea.isnull().sum()

pk                        0
old_addr_idx              0
dong                      0
ho                        0
official_price            0
net_leasable_area         0
updated                   0
complex_name              0
시도                        0
시군구                       0
읍면동                    9343
리                    412322
호                         0
호int                      0
호str                 407363
층                      7279
codes                     0
area                      0
위도                     9343
경도                     9343
dtype: int64

In [37]:
fill = korea[korea['위도'].isnull()==True]
fill

,pk,old_addr_idx,dong,ho,official_price,net_leasable_area,updated,complex_name,시도,시군구,읍면동,리,호,호int,호str,층,codes,평수,area,위도,경도
5371,00010906_0001_0001,3014010500_00022_00007_apartment,제1,1001,141000000,54.675,2021-01-01,현대1,대전광역시,중구,NaN,NaN,1001,1001,NaN,10,10.0,16.568182,17.0,NaN,NaN
5372,00010906_0001_0002,3014010500_00022_00007_apartment,제1,1002,141000000,54.675,2021-01-01,현대1,대전광역시,중구,NaN,NaN,1002,1002,NaN,10,10.0,16.568182,17.0,NaN,NaN
5373,00010906_0001_0003,3014010500_00022_00007_apartment,제1,1003,141000000,54.675,2021-01-01,현대1,대전광역시,중구,NaN,NaN,1003,1003,NaN,10,10.0,16.568182,17.0,NaN,NaN
5374,00010906_0001_0004,3014010500_00022_00007_apartment,제1,1004,141000000,54.675,2021-01-01,현대1,대전광역시,중구,NaN,NaN,1004,1004,NaN,10,10.0,16.568182,17.0,NaN,NaN
5375,00010906_0001_0005,3014010500_00022_00007_apartment,제1,1005,141000000,54.675,2021-01-01,현대1,대전광역시,중구,NaN,NaN,1005,1005,NaN,10,10.0,16.568182,17.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408871,20415197_0001_0004,3017010300_00124_00017_apartment,주건축물제1,304,117000000,46.500,2021-01-01,(124-17),대전광역시,서구,NaN,NaN,304,304,NaN,3,3.0,14.090909,14.0,NaN,NaN
408872,20415197_0001_0005,3017010300_00124_00017_apartment,주건축물제1,403,152000000,60.620,2021-01-01,(124-17),대전광역시,서구,NaN,NaN,403,403,NaN,4,4.0,18.369697,18.0,NaN,NaN
408873,20415197_0001_0006,3017010300_00124_00017_apartment,주건축물제1,404,117000000,46.500,2021-01-01,(124-17),대전광역시,서구,NaN,NaN,404,404,NaN,4,4.0,14.090909,14.0,NaN,NaN
408874,20415197_0001_0007,3017010300_00124_00017_apartment,주건축물제1,503,139000000,55.360,2021-01-01,(124-17),대전광역시,서구,NaN,NaN,503,503,NaN,5,5.0,16.775758,17.0,NaN,NaN


In [62]:
fill.isnull().sum()

pk                      0
old_addr_idx            0
dong                    0
ho                      0
official_price          0
net_leasable_area       0
updated                 0
complex_name            0
시도                      0
시군구                     0
읍면동                  9343
리                    9343
호                       0
호int                    0
호str                 8794
층                     549
codes                   0
area                    0
위도                   9343
경도                   9343
dtype: int64

In [38]:
code = daejeon_areaCode.groupby(['시도','시군구']).mean().reset_index()
code

<ipython-input-38-1d0cc780304f>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  code = daejeon_areaCode.groupby(['시도','시군구']).mean().reset_index()


,시도,시군구,위도,경도
0,대전광역시,대덕구,36.411416,127.432594
1,대전광역시,동구,36.330666,127.462753
2,대전광역시,서구,36.296259,127.353621
3,대전광역시,유성구,36.381529,127.345077
4,대전광역시,중구,36.301204,127.413932


In [39]:
code['읍면동'] = np.nan
code

,시도,시군구,위도,경도,읍면동
0,대전광역시,대덕구,36.411416,127.432594,NaN
1,대전광역시,동구,36.330666,127.462753,NaN
2,대전광역시,서구,36.296259,127.353621,NaN
3,대전광역시,유성구,36.381529,127.345077,NaN
4,대전광역시,중구,36.301204,127.413932,NaN


In [40]:
korea = code.set_index(['시도', '시군구','읍면동']).combine_first(korea.set_index(['시도', '시군구', '읍면동'])).reset_index()
korea

,시도,시군구,읍면동,area,codes,complex_name,dong,ho,net_leasable_area,official_price,old_addr_idx,pk,updated,경도,리,위도,층,평수,호,호int,호str
0,대전광역시,대덕구,대화동,22.0,1.0,금강,1,101,73.55,110000000,3023010200_00027_00007_apartment,00011718_0001_0001,2022-01-01,127.412043,NaN,36.365960,1,22.287879,101,101,NaN
1,대전광역시,대덕구,대화동,22.0,1.0,금강,1,102,73.55,110000000,3023010200_00027_00007_apartment,00011718_0001_0002,2022-01-01,127.412043,NaN,36.365960,1,22.287879,102,102,NaN
2,대전광역시,대덕구,대화동,17.0,1.0,금강,1,103,56.26,100000000,3023010200_00027_00007_apartment,00011718_0001_0003,2022-01-01,127.412043,NaN,36.365960,1,17.048485,103,103,NaN
3,대전광역시,대덕구,대화동,17.0,1.0,금강,1,104,56.26,100000000,3023010200_00027_00007_apartment,00011718_0001_0004,2022-01-01,127.412043,NaN,36.365960,1,17.048485,104,104,NaN
4,대전광역시,대덕구,대화동,17.0,1.0,금강,1,105,56.26,103000000,3023010200_00027_00007_apartment,00011718_0001_0005,2022-01-01,127.412043,NaN,36.365960,1,17.048485,105,105,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,대전광역시,중구,NaN,18.0,3.0,(645-10),1,302,60.28,78700000,3014011900_00645_00010_apartment,20293767_0001_0004,2021-01-01,127.413932,NaN,36.301204,3,18.266667,302,302,NaN
412318,대전광역시,중구,NaN,18.0,4.0,(645-10),1,401,60.28,78700000,3014011900_00645_00010_apartment,20293767_0001_0005,2021-01-01,127.413932,NaN,36.301204,4,18.266667,401,401,NaN
412319,대전광역시,중구,NaN,18.0,4.0,(645-10),1,402,60.28,78700000,3014011900_00645_00010_apartment,20293767_0001_0006,2021-01-01,127.413932,NaN,36.301204,4,18.266667,402,402,NaN
412320,대전광역시,중구,NaN,18.0,5.0,(645-10),1,501,60.28,78700000,3014011900_00645_00010_apartment,20293767_0001_0007,2021-01-01,127.413932,NaN,36.301204,5,18.266667,501,501,NaN


In [66]:
korea.isnull().sum()

시도                        0
시군구                       0
읍면동                    9343
area                      0
codes                     0
complex_name              0
dong                      0
ho                        0
net_leasable_area         0
official_price            0
old_addr_idx              0
pk                        0
updated                   0
경도                        0
리                    412322
위도                        0
층                      7279
호                         0
호int                      0
호str                 407363
dtype: int64

In [43]:
daejeon_match = korea[['pk', 'old_addr_idx', 'official_price', '시도', '시군구', '읍면동', '리', 'area', 'codes', 'complex_name', 'dong', 'ho', 'updated', '위도', '경도', '평수']]
daejeon_match = daejeon_match.rename(columns={'codes':'floor'})
daejeon_match = daejeon_match.drop_duplicates()

In [49]:
daejeon_match['위도+경도'] = daejeon_match['위도'].astype(str)+daejeon_match['경도'].astype(str)

In [50]:
daejeon_match

,pk,old_addr_idx,official_price,시도,시군구,읍면동,리,area,floor,complex_name,dong,ho,updated,위도,경도,평수,위도+경도
0,00011718_0001_0001,3023010200_00027_00007_apartment,110000000,대전광역시,대덕구,대화동,NaN,22.0,1.0,금강,1,101,2022-01-01,36.365960,127.412043,22.287879,36.36596127.412043
1,00011718_0001_0002,3023010200_00027_00007_apartment,110000000,대전광역시,대덕구,대화동,NaN,22.0,1.0,금강,1,102,2022-01-01,36.365960,127.412043,22.287879,36.36596127.412043
2,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1.0,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043
3,00011718_0001_0004,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1.0,금강,1,104,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043
4,00011718_0001_0005,3023010200_00027_00007_apartment,103000000,대전광역시,대덕구,대화동,NaN,17.0,1.0,금강,1,105,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412317,20293767_0001_0004,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,3.0,(645-10),1,302,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847
412318,20293767_0001_0005,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,4.0,(645-10),1,401,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847
412319,20293767_0001_0006,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,4.0,(645-10),1,402,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847
412320,20293767_0001_0007,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,5.0,(645-10),1,501,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847


In [59]:
df3 = pd.read_csv('/home/subin/다운로드/여피/0315 업무/trade_history_2018-2023_edit.csv', header=None)
df3.columns = ['old_addr_idx', '거래날짜', 'actual', '면적', 'floor', '모름', '거래방식', '모름2', '모름3', '시군구', '주소', '읍면동', 'complex_name', '모름4', '모름5', 'property_type']
df3 = df3.drop_duplicates() 
df3['area'] = (df3['면적']/3.3).round(0)
df3.fillna("NONE", inplace=True)
df3 = df3[~df3['property_type'].str.contains('officetel')]
df3 = df3[['old_addr_idx', 'actual', 'area', 'floor', '거래날짜', 'complex_name', '시군구', '주소', '읍면동']]
df3

<ipython-input-59-15c628132692>:1: DtypeWarning: Columns (5,6,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('/home/subin/다운로드/여피/0315 업무/trade_history_2018-2023_edit.csv', header=None)


,old_addr_idx,actual,area,floor,거래날짜,complex_name,시군구,주소,읍면동
0,3023010200_00034_00000_apartment,99000000,18.0,2,2020-02-28,NONE,NONE,NONE,NONE
1,3023010200_00027_00007_apartment,70000000,17.0,5,2020-02-03,NONE,NONE,NONE,NONE
2,3023010200_00027_00007_apartment,66000000,17.0,1,2020-03-22,NONE,NONE,NONE,NONE
3,3023010200_00027_00007_apartment,77000000,17.0,3,2020-04-29,NONE,NONE,NONE,NONE
4,3023010200_00027_00007_apartment,70000000,17.0,5,2020-05-25,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...
2203363,5011010400_01034_00001_apartment,280000000,25.0,5,2022-09-16,이도월드하우스,제주 제주시,1034-1,이도이동
2203364,5011010400_00777_00000_apartment,580000000,12.0,4,2022-09-27,주공3,제주 제주시,777,이도이동
2203365,5013010400_02161_00000_apartment,317000000,26.0,2,2022-09-01,호근동코아루오션뷰,제주 서귀포시,2161,호근동
2203366,5013010500_00147_00000_apartment,214000000,12.0,1,2022-09-01,주공1,제주 서귀포시,147,동홍동


In [71]:
df3['area'] = df3['area'].round()
df3

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
0,3023010200_00034_00000_apartment,99000000,18.0,2,2020-02-28,NONE,2020-02-28,NONE,NONE,NONE
1,3023010200_00027_00007_apartment,70000000,17.0,5,2020-02-03,NONE,2020-02-03,NONE,NONE,NONE
2,3023010200_00027_00007_apartment,66000000,17.0,1,2020-03-22,NONE,2020-03-22,NONE,NONE,NONE
3,3023010200_00027_00007_apartment,77000000,17.0,3,2020-04-29,NONE,2020-04-29,NONE,NONE,NONE
4,3023010200_00027_00007_apartment,70000000,17.0,5,2020-05-25,NONE,2020-05-25,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...
2203363,5011010400_01034_00001_apartment,280000000,25.0,5,2022-09-16,이도월드하우스,2022-09-16,제주 제주시,1034-1,이도이동
2203364,5011010400_00777_00000_apartment,580000000,12.0,4,2022-09-27,주공3,2022-09-27,제주 제주시,777,이도이동
2203365,5013010400_02161_00000_apartment,317000000,26.0,2,2022-09-01,호근동코아루오션뷰,2022-09-01,제주 서귀포시,2161,호근동
2203366,5013010500_00147_00000_apartment,214000000,12.0,1,2022-09-01,주공1,2022-09-01,제주 서귀포시,147,동홍동


In [60]:
actual_match = df3[['old_addr_idx', 'actual', 'area', 'floor', '거래날짜']]
actual_match = actual_match.drop_duplicates()
actual_match

,old_addr_idx,actual,area,floor,거래날짜
0,3023010200_00034_00000_apartment,99000000,18.0,2,2020-02-28
1,3023010200_00027_00007_apartment,70000000,17.0,5,2020-02-03
2,3023010200_00027_00007_apartment,66000000,17.0,1,2020-03-22
3,3023010200_00027_00007_apartment,77000000,17.0,3,2020-04-29
4,3023010200_00027_00007_apartment,70000000,17.0,5,2020-05-25
...,...,...,...,...,...
2203363,5011010400_01034_00001_apartment,280000000,25.0,5,2022-09-16
2203364,5011010400_00777_00000_apartment,580000000,12.0,4,2022-09-27
2203365,5013010400_02161_00000_apartment,317000000,26.0,2,2022-09-01
2203366,5013010500_00147_00000_apartment,214000000,12.0,1,2022-09-01


In [54]:
actual_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1968927 entries, 0 to 2206006
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   old_addr_idx  object 
 1   actual        int64  
 2   area          float64
 3   floor         int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 75.1+ MB


In [55]:
daejeon_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412322 entries, 0 to 412321
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pk              412322 non-null  object 
 1   old_addr_idx    412322 non-null  object 
 2   official_price  412322 non-null  int64  
 3   시도              412322 non-null  object 
 4   시군구             412322 non-null  object 
 5   읍면동             402979 non-null  object 
 6   리               0 non-null       float64
 7   area            412322 non-null  float64
 8   floor           412322 non-null  object 
 9   complex_name    412322 non-null  object 
 10  dong            412322 non-null  object 
 11  ho              412322 non-null  object 
 12  updated         412322 non-null  object 
 13  위도              412322 non-null  float64
 14  경도              412322 non-null  float64
 15  평수              412322 non-null  float64
 16  위도+경도           412322 non-null  object 
dtypes: float64

In [56]:
daejeon_match['floor'] = daejeon_match['floor'].astype(int)
daejeon_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412322 entries, 0 to 412321
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pk              412322 non-null  object 
 1   old_addr_idx    412322 non-null  object 
 2   official_price  412322 non-null  int64  
 3   시도              412322 non-null  object 
 4   시군구             412322 non-null  object 
 5   읍면동             402979 non-null  object 
 6   리               0 non-null       float64
 7   area            412322 non-null  float64
 8   floor           412322 non-null  int64  
 9   complex_name    412322 non-null  object 
 10  dong            412322 non-null  object 
 11  ho              412322 non-null  object 
 12  updated         412322 non-null  object 
 13  위도              412322 non-null  float64
 14  경도              412322 non-null  float64
 15  평수              412322 non-null  float64
 16  위도+경도           412322 non-null  object 
dtypes: float64

In [61]:
daejeon_total = daejeon_match.merge(actual_match, on=['old_addr_idx', 'area','floor'], how='left')
daejeon_total

,pk,old_addr_idx,official_price,시도,시군구,읍면동,리,area,floor,complex_name,dong,ho,updated,위도,경도,평수,위도+경도,actual,거래날짜
0,00011718_0001_0001,3023010200_00027_00007_apartment,110000000,대전광역시,대덕구,대화동,NaN,22.0,1,금강,1,101,2022-01-01,36.365960,127.412043,22.287879,36.36596127.412043,78000000.0,2018-07-06
1,00011718_0001_0002,3023010200_00027_00007_apartment,110000000,대전광역시,대덕구,대화동,NaN,22.0,1,금강,1,102,2022-01-01,36.365960,127.412043,22.287879,36.36596127.412043,78000000.0,2018-07-06
2,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043,66000000.0,2020-03-22
3,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043,98000000.0,2020-08-18
4,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043,50000000.0,2018-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485315,20293767_0001_0004,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,3,(645-10),1,302,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN
2485316,20293767_0001_0005,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,4,(645-10),1,401,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN
2485317,20293767_0001_0006,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,4,(645-10),1,402,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN
2485318,20293767_0001_0007,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,5,(645-10),1,501,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN


In [62]:
daejeon_total.to_csv('대전광역시 공시지가&실거래가 통합본.csv')

In [63]:
school = pd.read_csv('/home/subin/MyGitProject/hsj-14/대전광역시 학교 거리_이후버전.csv')

In [64]:
school['school_id'] = school['school_id'].replace('인천과학예술영재학교', '인천과학예술영재고등학교')
school['school_id'] = school['school_id'].replace('동성고등공민학교', '동성공민고등학교')

In [69]:
school['school_id'] = school['school_id'].replace('세종과학예술영재학교', '세종과학예술영재고등학교')
school['school_id'] = school['school_id'].replace('진북고등공민학교', '진북공민고등학교')
school['school_id'] = school['school_id'].replace('KIS High School', 'KIS 고등학교')

In [68]:
needAdjust = school[~school['school_id'].str.contains('초등학교|중학교|고등학교')]
needAdjust['school_id'].value_counts()

세인트존스베리 아카데미 제주        333
세종과학예술영재학교             111
NLCS JeJu              111
진북고등공민학교               111
KIS High School        111
Branksome Hall Asia    111
한국국제학교                 111
한국과학영재학교               111
Name: school_id, dtype: int64

In [92]:
check_sigungu = app_apt[app_apt['시군구']=='대덕구']
check_sigungu['시도'].value_counts()

대전광역시    52839
Name: 시도, dtype: int64

In [70]:
daejeon_final = pd.read_csv('/home/subin/MyGitProject/hsj-14/대전광역시 전처리 최종.csv')

<ipython-input-70-898b392c7cdc>:1: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  daejeon_final = pd.read_csv('/home/subin/MyGitProject/hsj-14/대전광역시 전처리 최종.csv')


In [72]:
daejeon_final.isnull().sum()

pk                      0
old_addr_idx            0
official_price          0
시도                      0
시군구                     0
읍면동                  9343
리                 2485320
area                    0
floor                   0
complex_name            0
dong                    0
ho                      0
updated                 0
위도                      0
경도                      0
평수                      0
위도+경도                   0
actual             143063
거래날짜               143063
school_1km              0
closest_high            0
closest_sub             0
dtype: int64

In [71]:
daejeon_final

,pk,old_addr_idx,official_price,시도,시군구,읍면동,리,area,floor,complex_name,dong,ho,updated,위도,경도,평수,위도+경도,actual,거래날짜,school_1km,closest_high,closest_sub
0,00011718_0001_0001,3023010200_00027_00007_apartment,110000000,대전광역시,대덕구,대화동,NaN,22.0,1,금강,1,101,2022-01-01,36.365960,127.412043,22.287879,36.36596127.412043,78000000.0,2018-07-06,3.0,1.495271,4.114766
1,00011718_0001_0002,3023010200_00027_00007_apartment,110000000,대전광역시,대덕구,대화동,NaN,22.0,1,금강,1,102,2022-01-01,36.365960,127.412043,22.287879,36.36596127.412043,78000000.0,2018-07-06,3.0,1.495271,4.114766
2,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043,66000000.0,2020-03-22,3.0,1.495271,4.114766
3,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043,98000000.0,2020-08-18,3.0,1.495271,4.114766
4,00011718_0001_0003,3023010200_00027_00007_apartment,100000000,대전광역시,대덕구,대화동,NaN,17.0,1,금강,1,103,2022-01-01,36.365960,127.412043,17.048485,36.36596127.412043,50000000.0,2018-01-27,3.0,1.495271,4.114766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485315,20293767_0001_0004,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,3,(645-10),1,302,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN,0.0,1.516935,2.551867
2485316,20293767_0001_0005,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,4,(645-10),1,401,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN,0.0,1.516935,2.551867
2485317,20293767_0001_0006,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,4,(645-10),1,402,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN,0.0,1.516935,2.551867
2485318,20293767_0001_0007,3014011900_00645_00010_apartment,78700000,대전광역시,중구,NaN,NaN,18.0,5,(645-10),1,501,2021-01-01,36.301204,127.413932,18.266667,36.3012045127.41393246153847,NaN,NaN,0.0,1.516935,2.551867


In [ ]:
대덕구	36.411416	127.432594	NaN
1	대전광역시	동구	36.330666	127.462753	NaN
2	대전광역시	서구	36.296259	127.353621	NaN
3	대전광역시	유성구	36.381529	127.345077	NaN
4	대전광역시	중구	36.301204

DELETE FROM off_act_price WHERE 시도='대전광역시' ORDER BY 시도

In [79]:
daejeon_total.isnull().sum()

pk                      0
old_addr_idx            0
official_price          0
시도                      0
시군구                     0
읍면동                  9343
리                 2271812
area                    0
floor                   0
complex_name            0
dong                    0
ho                      0
updated                 0
위도                      0
경도                      0
actual             143063
dtype: int64

In [57]:
daejeon_match_more = daejeon_match[['old_addr_idx', 'official_price', 'area', 'floor']]
daejeon_match_more.drop_duplicates(inplace=True)

<ipython-input-57-3e0457491344>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daejeon_match_more.drop_duplicates(inplace=True)


In [59]:
daejeon_match_more

,old_addr_idx,official_price,area,floor
0,3014010200_00160_00001_apartment,93800000,42.0,2.0
1,3014010200_00160_00001_apartment,92800000,42.0,2.0
2,3014010200_00160_00001_apartment,67300000,28.0,2.0
7,3014010200_00160_00001_apartment,60500000,26.0,2.0
8,3014010200_00160_00001_apartment,92100000,41.0,3.0
...,...,...,...,...
412282,3011011700_00135_00002_apartment,74500000,18.0,1.0
412290,3011011700_00135_00002_apartment,77300000,18.0,2.0
412298,3011011700_00135_00002_apartment,81900000,18.0,3.0
412306,3011011700_00135_00002_apartment,77300000,18.0,4.0


In [117]:
pd.set_option('display.max_rows', 10)

In [116]:
# 이전 버전
pd.set_option('display.max_rows', None)
daejeon[daejeon['old_addr_idx']=='3014010200_00160_00001_apartment']

,pk,old_addr_idx,dong,ho,official_price,updated,complex_name,시도,시군구,읍면동,리,평수
3032783,00010892_0001_0001,3014010200_00160_00001_apartment,단일,201,93800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,42.500000
3032784,00010892_0001_0002,3014010200_00160_00001_apartment,단일,202,92800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,41.733333
3032785,00010892_0001_0003,3014010200_00160_00001_apartment,단일,203,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032786,00010892_0001_0004,3014010200_00160_00001_apartment,단일,204,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032787,00010892_0001_0005,3014010200_00160_00001_apartment,단일,205,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032788,00010892_0001_0006,3014010200_00160_00001_apartment,단일,206,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.100000
3032789,00010892_0001_0007,3014010200_00160_00001_apartment,단일,207,67300000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,28.448485
3032790,00010892_0001_0008,3014010200_00160_00001_apartment,단일,208,60500000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,26.090909
3032791,00010892_0001_0009,3014010200_00160_00001_apartment,단일,301,92100000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,40.654545
3032792,00010892_0001_0010,3014010200_00160_00001_apartment,단일,302,90800000,2022-01-01,선화맨션,대전광역시,중구,선화동,NaN,39.881818


In [114]:
df3[df3['old_addr_idx']=='3014010200_00160_00001_apartment']

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
23340,3014010200_00160_00001_apartment,82000000,24.683333,4,2020-05-21,NaN,2020-05-21,NaN,NaN,NaN
44278,3014010200_00160_00001_apartment,75000000,26.878788,3,2018-03-09,NaN,2018-03-09,NaN,NaN,NaN
44279,3014010200_00160_00001_apartment,60000000,24.683333,4,2018-04-02,NaN,2018-04-02,NaN,NaN,NaN
44280,3014010200_00160_00001_apartment,27500000,18.166667,6,2018-07-26,NaN,2018-07-26,NaN,NaN,NaN
59196,3014010200_00160_00001_apartment,68000000,24.683333,4,2021-01-18,NaN,2021-01-18,NaN,NaN,NaN
59197,3014010200_00160_00001_apartment,86000000,24.683333,4,2021-08-03,NaN,2021-08-03,NaN,NaN,NaN
82567,3014010200_00160_00001_apartment,82000000,26.878788,3,2019-06-28,NaN,2019-06-28,NaN,NaN,NaN


In [3]:
df3 = pd.read_csv('/home/subin/다운로드/여피/0315 업무/trade_history_2018-2023_edit.csv', header=None)
df3.columns = ['old_addr_idx', '거래날짜', 'actual', '면적', 'floor', '모름', '거래방식', '모름2', '모름3', '시군구', '주소', '읍면동', 'complex_name', '모름4', '모름5', 'property_type']
df3 = df3.drop_duplicates() 
df3['area'] = df3['면적']/3.3
df3.fillna("NONE", inplace=True)
df3 = df3[~df3['property_type'].str.contains('officetel')]
df3 = df3[['old_addr_idx', 'actual', 'area', 'floor', '거래날짜', 'complex_name', '거래날짜', '시군구', '주소', '읍면동']]

<ipython-input-3-3d74f6f0beb1>:1: DtypeWarning: Columns (5,6,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('/home/subin/다운로드/여피/0315 업무/trade_history_2018-2023_edit.csv', header=None)


In [4]:
df3 = df3.replace('NONE', np.nan)
df3.isnull().sum()

old_addr_idx          0
actual                0
area                  0
floor                 0
거래날짜                  0
complex_name    2136478
거래날짜                  0
시군구             2137611
주소              2136478
읍면동             2136478
dtype: int64

In [5]:
df3

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
0,3023010200_00034_00000_apartment,99000000,17.890909,2,2020-02-28,NaN,2020-02-28,NaN,NaN,NaN
1,3023010200_00027_00007_apartment,70000000,17.048485,5,2020-02-03,NaN,2020-02-03,NaN,NaN,NaN
2,3023010200_00027_00007_apartment,66000000,17.048485,1,2020-03-22,NaN,2020-03-22,NaN,NaN,NaN
3,3023010200_00027_00007_apartment,77000000,17.048485,3,2020-04-29,NaN,2020-04-29,NaN,NaN,NaN
4,3023010200_00027_00007_apartment,70000000,17.048485,5,2020-05-25,NaN,2020-05-25,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2203363,5011010400_01034_00001_apartment,280000000,25.436970,5,2022-09-16,이도월드하우스,2022-09-16,제주 제주시,1034-1,이도이동
2203364,5011010400_00777_00000_apartment,580000000,12.027273,4,2022-09-27,주공3,2022-09-27,제주 제주시,777,이도이동
2203365,5013010400_02161_00000_apartment,317000000,25.704212,2,2022-09-01,호근동코아루오션뷰,2022-09-01,제주 서귀포시,2161,호근동
2203366,5013010500_00147_00000_apartment,214000000,12.027273,1,2022-09-01,주공1,2022-09-01,제주 서귀포시,147,동홍동


In [8]:
address = df1[['old_addr_idx', '시도', '시군구', '읍면동']]

In [9]:
address = address.drop_duplicates()

In [10]:
address

,old_addr_idx,시도,시군구,읍면동
0,1111010100_00001_00000_apartment,서울특별시,종로구,청운동
1,1111010100_00003_00000_apartment,서울특별시,종로구,청운동
2,1111010100_00003_00150_apartment,서울특별시,종로구,청운동
3,1111010100_00004_00001_apartment,서울특별시,종로구,청운동
4,1111010100_00004_00003_apartment,서울특별시,종로구,청운동
...,...,...,...,...
320920,5013032022_00048_00003_apartment,제주특별자치도,서귀포시,표선면
320921,5013032022_02167_00000_apartment,제주특별자치도,서귀포시,표선면
320922,5013032023_02008_00000_apartment,제주특별자치도,서귀포시,표선면
320923,5013032024_02521_00005_apartment,제주특별자치도,서귀포시,표선면


In [11]:
address['시군구_real'] = address['시군구']
address = address.drop('시군구', axis=1)

In [13]:
actWithAddress = df3.merge(address, on='old_addr_idx', how='left')

In [14]:
actWithAddress

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동_x,시도,읍면동_y,시군구_real
0,3023010200_00034_00000_apartment,99000000,17.890909,2,2020-02-28,NaN,2020-02-28,NaN,NaN,NaN,대전광역시,대화동,대덕구
1,3023010200_00027_00007_apartment,70000000,17.048485,5,2020-02-03,NaN,2020-02-03,NaN,NaN,NaN,대전광역시,대화동,대덕구
2,3023010200_00027_00007_apartment,66000000,17.048485,1,2020-03-22,NaN,2020-03-22,NaN,NaN,NaN,대전광역시,대화동,대덕구
3,3023010200_00027_00007_apartment,77000000,17.048485,3,2020-04-29,NaN,2020-04-29,NaN,NaN,NaN,대전광역시,대화동,대덕구
4,3023010200_00027_00007_apartment,70000000,17.048485,5,2020-05-25,NaN,2020-05-25,NaN,NaN,NaN,대전광역시,대화동,대덕구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2141152,5011010400_01034_00001_apartment,280000000,25.436970,5,2022-09-16,이도월드하우스,2022-09-16,제주 제주시,1034-1,이도이동,제주특별자치도,이도이동,제주시
2141153,5011010400_00777_00000_apartment,580000000,12.027273,4,2022-09-27,주공3,2022-09-27,제주 제주시,777,이도이동,제주특별자치도,이도이동,제주시
2141154,5013010400_02161_00000_apartment,317000000,25.704212,2,2022-09-01,호근동코아루오션뷰,2022-09-01,제주 서귀포시,2161,호근동,제주특별자치도,호근동,서귀포시
2141155,5013010500_00147_00000_apartment,214000000,12.027273,1,2022-09-01,주공1,2022-09-01,제주 서귀포시,147,동홍동,제주특별자치도,동홍동,서귀포시


In [15]:
actWithAddress.isnull().sum()

old_addr_idx          0
actual                0
area                  0
floor                 0
거래날짜                  0
complex_name    2136478
거래날짜                  0
시군구             2137611
주소              2136478
읍면동_x           2136478
시도                   95
읍면동_y               215
시군구_real            107
dtype: int64

In [23]:
sidolist = address['sido'].unique().tolist()

In [105]:
k=5
each = address[address['sido']==sidolist[k]]
each

,old_addr_idx,sido,sigungu,eubmyeondong,li
178887,3011010100_00035_00003_apartment,대전광역시,동구,원동,NaN
178888,3011010200_00012_00005_apartment,대전광역시,동구,인동,NaN
178890,3011010200_00014_00001_apartment,대전광역시,동구,인동,NaN
178891,3011010200_00018_00001_apartment,대전광역시,동구,인동,NaN
178892,3011010200_00029_00002_apartment,대전광역시,동구,인동,NaN
...,...,...,...,...,...
184117,3023012600_00264_00046_apartment,대전광역시,대덕구,신탄진동,NaN
184118,3023012600_00264_00069_apartment,대전광역시,대덕구,신탄진동,NaN
184119,3023012600_00295_00293_apartment,대전광역시,대덕구,신탄진동,NaN
184120,3023012600_00451_00000_apartment,대전광역시,대덕구,신탄진동,NaN


In [30]:
check_act = act_name.merge(address, on='old_addr_idx', how='outer')
check_act

,old_addr_idx,sido,sigungu,eubmyeondong,li
0,3023010200_00034_00000_apartment,대전광역시,대덕구,대화동,NaN
1,3023010200_00027_00007_apartment,대전광역시,대덕구,대화동,NaN
2,3023010200_00016_00002_apartment,대전광역시,대덕구,대화동,NaN
3,3023010200_00040_00067_apartment,대전광역시,대덕구,대화동,NaN
4,3023010200_00031_00036_apartment,대전광역시,대덕구,대화동,NaN
...,...,...,...,...,...
306017,5013032022_00048_00003_apartment,제주특별자치도,서귀포시,표선면,하천리
306018,5013032022_02167_00000_apartment,제주특별자치도,서귀포시,표선면,하천리
306019,5013032023_02008_00000_apartment,제주특별자치도,서귀포시,표선면,성읍리
306020,5013032024_02521_00005_apartment,제주특별자치도,서귀포시,표선면,가시리


In [79]:
not_exist_off = check_act[check_act['sido'].isnull()==True]
not_exist_off

,old_addr_idx,sido,sigungu,eubmyeondong,li
34949,1121510300_00680_00000_apartment,NaN,NaN,NaN,NaN
34956,1129013300_00150_00027_apartment,NaN,NaN,NaN,NaN
34962,1141011000_00149_00000_apartment,NaN,NaN,NaN,NaN
34972,1168010100_00902_00000_apartment,NaN,NaN,NaN,NaN
34982,2620010100_00176_00000_apartment,NaN,NaN,NaN,NaN
...,...,...,...,...,...
35660,1159010700_01160_00000_apartment,NaN,NaN,NaN,NaN
35663,2917010100_00836_00000_apartment,NaN,NaN,NaN,NaN
35667,4146310200_00661_00000_apartment,NaN,NaN,NaN,NaN
35679,4874025021_00053_00000_apartment,NaN,NaN,NaN,NaN


In [82]:
not_exist_off_list = not_exist_off['old_addr_idx'].unique().tolist()

In [88]:
not_exist_off_df = df3[df3['old_addr_idx']==not_exist_off_list[0]]

i=1
while i<len(not_exist_off_list):
    for_concat = df3[df3['old_addr_idx']==not_exist_off_list[i]]
    not_exist_off_df = pd.concat([not_exist_off_df, for_concat])
    i=i+1
not_exist_off_df

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동
2184966,1121510300_00680_00000_apartment,1600000000,35.135394,24,2022-10-23,e편한세상광진그랜드파크,2022-10-23,NaN,680,구의동
2184993,1129013300_00150_00027_apartment,193000000,3.893939,10,2022-10-03,갤럭시더정릉,2022-10-03,NaN,150-27,정릉동
2185033,1141011000_00149_00000_apartment,1240000000,18.136364,2,2022-10-29,e편한세상신촌3단지,2022-10-29,서울 서대문구,149,북아현동
2200659,1141011000_00149_00000_apartment,1100000000,18.136364,17,2022-11-05,e편한세상신촌2단지,2022-11-05,서울 서대문구,149,북아현동
2185118,1168010100_00902_00000_apartment,3000000000,25.731818,8,2022-10-06,강남센트럴아이파크,2022-10-06,"서울 강남구, 서울 송파구",902,역삼동
...,...,...,...,...,...,...,...,...,...,...
2200689,1159010700_01160_00000_apartment,895000000,18.179030,12,2022-11-27,사당롯데캐슬골든포레,2022-11-27,서울 동작구,1160,사당동
2202760,2917010100_00836_00000_apartment,870000000,33.269697,13,2022-10-30,제일풍경채센트럴파크1단지,2022-10-30,광주 북구,836,중흥동
2202775,4146310200_00661_00000_apartment,800000000,25.751818,33,2022-10-04,기흥역더샵,2022-10-04,NaN,661,구갈동
2203333,4874025021_00053_00000_apartment,64000000,18.163636,5,2022-09-26,도원,2022-09-26,경남 창녕군,53,창녕읍 송현리


In [93]:
not_exist_off_df['시도'] = not_exist_off_df['시군구'].str.split(' ').str[0]

In [95]:
not_exist_off_df

,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜,시군구,주소,읍면동,시도
2184966,1121510300_00680_00000_apartment,1600000000,35.135394,24,2022-10-23,e편한세상광진그랜드파크,2022-10-23,NaN,680,구의동,NaN
2184993,1129013300_00150_00027_apartment,193000000,3.893939,10,2022-10-03,갤럭시더정릉,2022-10-03,NaN,150-27,정릉동,NaN
2185033,1141011000_00149_00000_apartment,1240000000,18.136364,2,2022-10-29,e편한세상신촌3단지,2022-10-29,서울 서대문구,149,북아현동,서울
2200659,1141011000_00149_00000_apartment,1100000000,18.136364,17,2022-11-05,e편한세상신촌2단지,2022-11-05,서울 서대문구,149,북아현동,서울
2185118,1168010100_00902_00000_apartment,3000000000,25.731818,8,2022-10-06,강남센트럴아이파크,2022-10-06,"서울 강남구, 서울 송파구",902,역삼동,서울
...,...,...,...,...,...,...,...,...,...,...,...
2200689,1159010700_01160_00000_apartment,895000000,18.179030,12,2022-11-27,사당롯데캐슬골든포레,2022-11-27,서울 동작구,1160,사당동,서울
2202760,2917010100_00836_00000_apartment,870000000,33.269697,13,2022-10-30,제일풍경채센트럴파크1단지,2022-10-30,광주 북구,836,중흥동,광주
2202775,4146310200_00661_00000_apartment,800000000,25.751818,33,2022-10-04,기흥역더샵,2022-10-04,NaN,661,구갈동,NaN
2203333,4874025021_00053_00000_apartment,64000000,18.163636,5,2022-09-26,도원,2022-09-26,경남 창녕군,53,창녕읍 송현리,경남


In [91]:
pd.set_option('display.max_rows', 10)

In [96]:
not_exist_off_df.to_csv('공시지가에 없는 실거래가 아파트 코드.csv')

In [98]:
read = pd.read_csv('/home/subin/MyGitProject/hsj-14/공시지가에 없는 실거래가 아파트 코드.csv')
read

,Unnamed: 0,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜.1,시군구,주소,읍면동,시도
0,2184966,1121510300_00680_00000_apartment,1600000000,35.135394,24,2022-10-23,e편한세상광진그랜드파크,2022-10-23,NaN,680,구의동,서울
1,2184993,1129013300_00150_00027_apartment,193000000,3.893939,10,2022-10-03,갤럭시더정릉,2022-10-03,NaN,150-27,정릉동,서울
2,2185033,1141011000_00149_00000_apartment,1240000000,18.136364,2,2022-10-29,e편한세상신촌3단지,2022-10-29,서울 서대문구,149,북아현동,서울
3,2200659,1141011000_00149_00000_apartment,1100000000,18.136364,17,2022-11-05,e편한세상신촌2단지,2022-11-05,서울 서대문구,149,북아현동,서울
4,2185118,1168010100_00902_00000_apartment,3000000000,25.731818,8,2022-10-06,강남센트럴아이파크,2022-10-06,"서울 강남구, 서울 송파구",902,역삼동,서울
...,...,...,...,...,...,...,...,...,...,...,...,...
90,2200689,1159010700_01160_00000_apartment,895000000,18.179030,12,2022-11-27,사당롯데캐슬골든포레,2022-11-27,서울 동작구,1160,사당동,서울
91,2202760,2917010100_00836_00000_apartment,870000000,33.269697,13,2022-10-30,제일풍경채센트럴파크1단지,2022-10-30,광주 북구,836,중흥동,광주
92,2202775,4146310200_00661_00000_apartment,800000000,25.751818,33,2022-10-04,기흥역더샵,2022-10-04,NaN,661,구갈동,경기
93,2203333,4874025021_00053_00000_apartment,64000000,18.163636,5,2022-09-26,도원,2022-09-26,경남 창녕군,53,창녕읍 송현리,경남


In [99]:
read[read['시도']=='대전']

,Unnamed: 0,old_addr_idx,actual,area,floor,거래날짜,complex_name,거래날짜.1,시군구,주소,읍면동,시도
59,2192458,3014010300_00001_00095_apartment,620000000,25.755091,26,2022-08-03,목동더샵리슈빌,2022-08-03,대전 중구,1-95,목동,대전
66,2197556,3014010200_00379_00001_apartment,229120000,14.999606,7,2022-06-25,에이스퀘어,2022-06-25,대전 중구,379-1,선화동,대전


In [31]:
check_act.isnull().sum()

old_addr_idx         0
sido                94
sigungu            493
eubmyeondong     11003
li              266222
dtype: int64

In [27]:
check_each.notnull().sum()

old_addr_idx    35683
sido             7529
sigungu          7529
eubmyeondong     7525
li                100
dtype: int64

In [15]:
address['sido'].value_counts()

서울특별시      105668
경기도         82801
인천광역시       26573
부산광역시       24201
경상남도         9310
대구광역시        9197
경상북도         9099
충청남도         6277
제주특별자치도      5778
대전광역시        5098
울산광역시        4684
충청북도         4086
전라북도         3549
강원도          3430
전라남도         3350
광주광역시        2419
세종특별자치시       408
Name: sido, dtype: int64

In [56]:
act_code = df3[['old_addr_idx']].drop_duplicates()

In [57]:
act_code

,old_addr_idx
0,3023010200_00034_00000_apartment
1,3023010200_00027_00007_apartment
9,3023010200_00016_00002_apartment
35,3023010200_00040_00067_apartment
41,3023010200_00031_00036_apartment
...,...
2203326,4833010200_00565_00001_apartment
2203333,4874025021_00053_00000_apartment
2203338,4882025021_00048_00012_apartment
2203346,4886037021_00601_00000_apartment


In [48]:
act_code_withName = df3[['old_addr_idx', 'complex_name']].drop_duplicates()

In [50]:
act_code_withName = act_code_withName.replace('NONE', np.nan)

In [51]:
act_code_withName

,old_addr_idx,complex_name
0,3023010200_00034_00000_apartment,NaN
1,3023010200_00027_00007_apartment,NaN
9,3023010200_00016_00002_apartment,NaN
35,3023010200_00040_00067_apartment,NaN
41,3023010200_00031_00036_apartment,NaN
...,...,...
2203363,5011010400_01034_00001_apartment,이도월드하우스
2203364,5011010400_00777_00000_apartment,주공3
2203365,5013010400_02161_00000_apartment,호근동코아루오션뷰
2203366,5013010500_00147_00000_apartment,주공1


In [54]:
39624-35683

3941

In [52]:
act_name = act_name.replace('NONE', np.nan)
act_name = act_code_withName.dropna(subset='complex_name').drop_duplicates()

In [53]:
act_name

,old_addr_idx,complex_name
2184937,1111017500_00202_00003_apartment,종로중흥S클래스
2184938,1111017500_00318_00002_apartment,삼전솔하임4차
2184939,1111017500_00296_00019_apartment,삼전솔하임2차
2184940,1114015400_00145_00001_apartment,오렌지카운티을지로
2184941,1114016200_00432_01666_apartment,남산FORESTHOUSE
...,...,...
2203363,5011010400_01034_00001_apartment,이도월드하우스
2203364,5011010400_00777_00000_apartment,주공3
2203365,5013010400_02161_00000_apartment,호근동코아루오션뷰
2203366,5013010500_00147_00000_apartment,주공1


In [65]:
a = act_name.drop('complex_name', axis=1)

In [72]:
a

,old_addr_idx
2184937,1111017500_00202_00003_apartment
2184938,1111017500_00318_00002_apartment
2184939,1111017500_00296_00019_apartment
2184940,1114015400_00145_00001_apartment
2184941,1114016200_00432_01666_apartment
...,...
2203363,5011010400_01034_00001_apartment
2203364,5011010400_00777_00000_apartment
2203365,5013010400_02161_00000_apartment
2203366,5013010500_00147_00000_apartment


In [71]:
a.drop_duplicates()

,old_addr_idx
2184937,1111017500_00202_00003_apartment
2184938,1111017500_00318_00002_apartment
2184939,1111017500_00296_00019_apartment
2184940,1114015400_00145_00001_apartment
2184941,1114016200_00432_01666_apartment
...,...
2203363,5011010400_01034_00001_apartment
2203364,5011010400_00777_00000_apartment
2203365,5013010400_02161_00000_apartment
2203366,5013010500_00147_00000_apartment


In [68]:
b = a[a.duplicated()==True]

In [70]:
b

,old_addr_idx
2184990,1129010300_00609_00001_apartment
2185122,1168010300_00012_00000_apartment
2185196,2632010100_00810_00000_apartment
2185330,2814010300_00154_00000_apartment
2186474,4831010100_00224_00000_apartment
2188676,2811011800_00027_00107_apartment
2188695,2818510200_00353_00000_apartment
2188778,3171025032_01329_00002_apartment
2189013,4217010100_00794_00000_apartment
2189218,4623010600_01665_00000_apartment


In [73]:
c = b.merge(act_name, on='old_addr_idx', how='left')

In [74]:
c

,old_addr_idx,complex_name
0,1129010300_00609_00001_apartment,한신
1,1129010300_00609_00001_apartment,한진(609-1)
2,1168010300_00012_00000_apartment,삼익대청아파트
3,1168010300_00012_00000_apartment,성원대치2단지아파트
4,2632010100_00810_00000_apartment,금곡8단지주공
...,...,...
69,1159010700_00105_00000_apartment,극동(105)
70,1159010700_00105_00000_apartment,사당우성2
71,4146510100_00664_00000_apartment,동아
72,4146510100_00664_00000_apartment,풍림아파트


In [69]:
b.shape

(30, 1)

In [58]:
act_code_merge_name = act_code.merge(act_name, on='old_addr_idx', how='left')

In [59]:
act_code_merge_name

,old_addr_idx,complex_name
0,3023010200_00034_00000_apartment,NaN
1,3023010200_00027_00007_apartment,NaN
2,3023010200_00016_00002_apartment,금성백조
3,3023010200_00040_00067_apartment,대화
4,3023010200_00031_00036_apartment,두레
...,...,...
35708,4833010200_00565_00001_apartment,양산신도시 벽산블루밍
35709,4874025021_00053_00000_apartment,도원
35710,4882025021_00048_00012_apartment,그네빌리지
35711,4886037021_00601_00000_apartment,MH미르젠아파트


In [19]:
check_act = act_name.merge(address, on='old_addr_idx', how='left')

In [20]:
check_act

,old_addr_idx,sido,sigungu,eubmyeondong,li
0,3023010200_00034_00000_apartment,대전광역시,대덕구,대화동,NaN
1,3023010200_00027_00007_apartment,대전광역시,대덕구,대화동,NaN
2,3023010200_00016_00002_apartment,대전광역시,대덕구,대화동,NaN
3,3023010200_00040_00067_apartment,대전광역시,대덕구,대화동,NaN
4,3023010200_00031_00036_apartment,대전광역시,대덕구,대화동,NaN
...,...,...,...,...,...
35678,4833010200_00565_00001_apartment,경상남도,양산시,남부동,NaN
35679,4874025021_00053_00000_apartment,NaN,NaN,NaN,NaN
35680,4882025021_00048_00012_apartment,경상남도,고성군,고성읍,성내리
35681,4886037021_00601_00000_apartment,NaN,NaN,NaN,NaN


In [21]:
check_act.isnull().sum()

old_addr_idx        0
sido               94
sigungu           106
eubmyeondong      214
li              29790
dtype: int64

In [22]:
check_act['sido'].value_counts()

서울특별시      7529
경기도        5974
부산광역시      3846
경상남도       3097
경상북도       2372
인천광역시      1840
대구광역시      1686
울산광역시      1328
전라북도       1102
전라남도       1046
충청남도       1039
강원도        1037
광주광역시       974
충청북도        974
제주특별자치도     921
대전광역시       812
세종특별자치시      12
Name: sido, dtype: int64